In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pickle
import time

from IPython.display import clear_output

from utils import get_views_coord
from utils import test_val_train
from utils import sample_conflict_timeline
from utils import get_hyper_priors
from utils import predict
from utils import plot_predictions
from utils import get_mse
from utils import get_metrics

import pymc3 as pm

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn import metrics

start_time = time.time()

Todo:

1) Make ready to run eksperiments on comuterome.   
    + What you could to loop over:
        > C for estimation (C for pred is given as all)
        > Kernel for short term trend
        > sb, ns, os and all
        > individuel mean function

----------------------


Q's) 
- why don't they use 1989 for train?

old note (maybe still relevatn):
So, you wnat to make it as a function that can by iterated over, shitching C and shortterm cov - and maybe even priors. It needs to give out mse and save it. You also need to be able to survey/plot the models you afterwards find most interesting.

remember you are only running on ged_sb_best. but that is an experiament perhaps to run...
- Do they exhipist different lenght scale?
- Are one easier to predict than others?
- Can they inform each other?
- Does it make any difference if we pool them?
->> But keep in mind how much you want wich this paper and that the main concerne is to make it comparebal to ViEWS; which uses sb in the presented paper..
-> But it would still be super nice to know, and to ahve in the appendix..

In [2]:
# 43.3 minutes with 148 time-lines
# C = 100 # -> 6 obs
# C = 64 # -> 12 obs
# C = 40 # -> 38 obs
# C = 32 # -> 60 obs
# C = 24 # -> 87 obs
# C = 16 # -> 158 obs

C_est = 100
C_pred = 64
conf_type = 'ged_best_sb' # 'ged_best_ns', 'ged_best_os', 'ged_best'

N=None
seed = 42
dem = False # not sure this is in use..

In [3]:
# get df:
df = get_views_coord()

# get train and validation id:
train_id, val_id = test_val_train(df)

# get train (train + val) and test id:
# train_id, test_id = test_val_train(df, test_time = True)

Train from 1/1990 trough 12/2011 (264)
Val from 1/2012 trough 12/2014 (36)
Test time from 1/2015 trough 12/2017 (36)
(Test=False, so test set not outputted)



In [4]:
#hyper_priors_dict = get_hyper_priors(plot = True)

In [5]:

hps = get_hyper_priors(plot = False)

with pm.Model() as model:

    # short term trend/irregularities ---------------------------------
    ℓ_s = pm.Gamma("ℓ_s", alpha=hps['ℓ_alpha_s'] , beta=hps['ℓ_beta_s'])
    η_s = pm.HalfCauchy("η_s", beta=hps['η_beta_s'])
    α_s = pm.Gamma("α_s", alpha=hps['α_alpha_s'], beta=hps['α_beta_s']) #for Rational Quadratic Kernel. Ignore for Quad or Matern 

    # mean and kernal for short term trend
    mean_s =  pm.gp.mean.Zero()
    #cov_s = η_s ** 2 * pm.gp.cov.RatQuad(1, ℓ_s, α_s) # this seems to help alot when you split the trends below
    cov_s = η_s ** 2 * pm.gp.cov.ExpQuad(1, ℓ_s) # this seems to help alot when you split the trends below
    # so this needs to be some thng you can vary over in a loop

    # GP short term trend 
    gp_s = pm.gp.Marginal(mean_func = mean_s, cov_func=cov_s)


    # long term trend -------------------------------------------------
    ℓ_l = pm.Gamma("ℓ_l", alpha=hps['ℓ_alpha_l'] , beta=hps['ℓ_beta_l'])
    η_l = pm.HalfCauchy("η_l", beta=hps['η_beta_l'])
    
    # mean and kernal for long term trend
    mean_l =  pm.gp.mean.Zero()
    cov_l = η_l **2 * pm.gp.cov.ExpQuad(1, ℓ_l) # Cov func.
    
    # GP short term trend 
    gp_l = pm.gp.Marginal(mean_func = mean_l, cov_func=cov_l)


    # noise (constant "white noise") -----------------------------------
    σ = pm.HalfCauchy("σ", beta=hps['σ_beta'])


    # sample and split X,y ---------------------------------------------  
    X, y, _, _ = sample_conflict_timeline(df = df, train_id = train_id, test_id = val_id, C = C_est, N = None)

    # Full GP ----------------------------------------------------------
    gp = gp_s + gp_l

    # sample:
    for i in range(y.shape[1]):
        
        print(f'Time-line {i+1}/{y.shape[1]} in the works...') 
        clear_output(wait=True)

        y_ = gp.marginal_likelihood(f'y_{i}', X=X[:,i][:,None], y=y[:,i], noise= σ)
        
    
    mp = pm.find_MAP()


map_df = pd.DataFrame({
        "Parameter": ["ℓ_s", "η_s", "α_s", "ℓ_l", "η_l", "σ"],
        "Value at MAP": [float(mp["ℓ_s"]), float(mp["η_s"]), float(mp["α_s"]), float(mp["ℓ_l"]), float(mp["η_l"]), float(mp["σ"])]
    })

In [6]:
map_df

,Parameter,Value at MAP
0,ℓ_s,3.260012
1,η_s,0.387174
2,α_s,4.000012
3,ℓ_l,37.015686
4,η_l,0.985967
5,σ,0.796994


In [7]:
end_time = time.time()
est_time = end_time - start_time

string = f'Run for {est_time/60:.3} minutes with {y.shape[1]} time-lines'
print(string)

Run for 1.56 minutes with 12 time-lines


In [8]:
df_new = predict(df = df, train_id = train_id, test_id = val_id, mp = mp, gp = gp, gp_s = gp_s, gp_l = gp_l, σ=σ, C=C_pred)

df_merged = pd.merge(df_new, df[['id', 'pg_id','year','gwcode', 'xcoord', 'ycoord','ged_best_sb','ged_best_ns', 'ged_best_os', 'ged_best']], how = 'left', on = 'id') # Should merge all but this is fine for now.

# god check, but not if you have many time lines
# plot_predictions(df_merged = df_merged)

mse_resutls_df, mse_dict = get_mse(df_merged = df_merged, train_id = train_id, test_id = val_id)


Time-line 158/158 in the works...


In [9]:
mse_resutls_df

,Gps,MSE insample (mean),MSE insample (sd),MSE outsample (mean),MSE outsample (sd)
0,Full,0.242694,0.246660,0.620046,0.900007
1,Short,0.376749,0.663254,0.738255,1.362951
2,long,0.308151,0.329019,0.622836,0.903078


# Classificaiton task 
### need to check to performance af a classsifier.
Stuff needs to get into functions and get nice and all that jazz

In [17]:

df_merged.sort_values(['pg_id', 'X'], inplace= True)
df_merged['mu_l_slope'] = df_merged.groupby('pg_id')['mu_l'].transform(np.gradient)
df_merged['mu_l_acc'] = df_merged.groupby('pg_id')['mu_l_slope'].transform(np.gradient)
df_merged['mu_l_mass'] = df_merged.groupby('pg_id')['mu_l'].transform(np.cumsum)


In [21]:
# def get_metrics(df_merged, train_id, test_id):

#     """A function that takes the merged df.
#     The df must now include both data from 'predict',
#     and the devrived slope, acc ans mass.
#     The function uses a simple rf classifier to test the temporal features.
#     Very simple classifier so results are only indicative"""

#     X_train = df_merged[df_merged['id'].isin(train_id)][['mu_l', 'mu_l_slope', 'mu_l_acc', 'mu_l_mass']] 
#     y_train = (df_merged[df_merged['id'].isin(train_id)]['y'] > 0) * 1

#     X_test = df_merged[df_merged['id'].isin(val_id)][['mu_l', 'mu_l_slope', 'mu_l_acc', 'mu_l_mass']] 
#     y_test = (df_merged[df_merged['id'].isin(val_id)]['y'] > 0) * 1

#     # totally vanilla - just indicative
#     model = RandomForestClassifier(n_estimators=100, max_depth=4, min_samples_split=8, random_state=42)
#     #model = AdaBoostClassifier(n_estimators=100, random_state=42)
#     #model = LogisticRegression()

#     model.fit(X_train, y_train)

#     y_train_pred = model.predict_proba(X_train)[:,1]
#     y_test_pred = model.predict_proba(X_test)[:,1]

#     AUC_train = metrics.roc_auc_score(y_train, y_train_pred)
#     AP_train = metrics.average_precision_score(y_train, y_train_pred)
#     BS_train = metrics.brier_score_loss(y_train, y_train_pred)

#     AUC_test = metrics.roc_auc_score(y_test, y_test_pred)
#     AP_test = metrics.average_precision_score(y_test, y_test_pred)
#     BS_test = metrics.brier_score_loss(y_test, y_test_pred)

#     df_results =  pd.DataFrame({
#             "Metrics": ["AUC", "AP", "BS"],
#             "Train": [AUC_train, AP_train, BS_train],
#             "Test": [AUC_test, AP_test, BS_test]
#         })

#     return(df_results)


In [22]:
df_results = get_metrics(df_merged = df_merged, train_id = train_id, test_id = val_id)

In [23]:
df_results

,Metrics,Train,Test
0,AUC,0.905609,0.662218
1,AP,0.511462,0.330632
2,BS,0.048210,0.097827


In [20]:
final_time = time.time()
final_run_time = final_time - start_time

string = f'Run for {final_run_time/60:.3} minutes'
print(string)

Run for 1.6e+02 minutes
